### Preparing Pre-labeled CSVs from UNSW for processing 

In [1]:
from label.parallel_label import *

In [2]:
column_names=['src_ip','src_port','dst_ip','dst_port','proto','state','duration','sbytes','dbytes',
'sttl','dttl','sloss','dloss','service','Sload','Dload','Spkts','Dpkts','swin','dwin','stcpb','dtcpb',
'smeansz','dmeansz','trans_depth','res_bdy_len','Sjit','Djit','timestamp','Ltime','Sintpkt','Dintpkt',
'tcprtt','synack','ackdat','is_sm_ips_ports','ct_state_ttl','ct_flw_http_mthd','is_ftp_login',
'ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ ltm','ct_src_dport_ltm',
'ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','Label']

Add column names to the csv of unsw. 

In [4]:
import os
import pandas as pd

def process_and_rename_columns(folder_path, new_column_names):
    # Ensure the column names list is not empty
    if not new_column_names:
        raise ValueError("New column names list cannot be empty.")

    # Iterate over all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path, header=None)  # Use header=None if there are no existing headers

                # Check if the number of new column names matches the number of columns in the CSV
                if len(new_column_names) != len(df.columns):
                    print(f"Skipping {file_path}: Number of new column names does not match number of columns.")
                    continue

                # Set the new column names
                df.columns = new_column_names

                # Save the DataFrame back to the same CSV file
                df.to_csv(file_path, index=False)  # Set index=False to avoid writing row numbers

                print(f"Processed {file_path}: Column names updated.")

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

folder_path = "/scratch/user/syedwali/Datasets/UNSW/provided_CSVs"  # Replace with the path to your folder
process_and_rename_columns(folder_path, column_names)


/tmp/job.11310885/ipykernel_63165/1816870050.py:15: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None)  # Use header=None if there are no existing headers


Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv: Column names updated.
Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv: Column names updated.


/tmp/job.11310885/ipykernel_63165/1816870050.py:15: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None)  # Use header=None if there are no existing headers


Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_2.csv: Column names updated.
Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_4.csv: Column names updated.


Adding the attack_category column with 'Benign' for entries where it was empty and the label was 0

In [22]:
import os
import pandas as pd

def process_and_update_csvs(folder_path):
    # Iterate over all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path, low_memory=False)
                
                # Check if 'attack_cat' column exists
                if 'attack_cat' in df.columns:
                    # Fill empty strings and NaN values in 'attack_cat' with 'Benign'
                    df['attack_cat'].replace('', 'Benign', inplace=True)  # Replace empty strings
                    df['attack_cat'].fillna('Benign', inplace=True)      # Replace NaN values

                    # Show value counts for 'attack_cat'
                    print(f"File: {file_path}")
                    print(df['attack_cat'].value_counts())

                    # Save the DataFrame back to the same CSV file
                    df.to_csv(file_path, index=False)  # Set index=False to avoid writing row numbers

                    print(f"Processed and updated {file_path}")

                else:
                    print(f"Skipping {file_path}: 'attack_cat' column not found.")

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

# Example usage
folder_path = "/scratch/user/syedwali/Datasets/UNSW/provided_CSVs"  # Replace with the path to your folder
process_and_update_csvs(folder_path)


File: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
attack_cat
Benign            677786
Generic             7522
Exploits            5409
 Fuzzers            5051
Reconnaissance      1759
DoS                 1167
Backdoors            534
Analysis             526
Shellcode            223
Worms                 24
Name: count, dtype: int64
Processed and updated /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
File: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv
attack_cat
Benign              542576
Generic             118198
Exploits             16574
 Fuzzers              9137
DoS                   5642
 Reconnaissance       5582
Analysis               873
Backdoor               759
 Shellcode             593
Worms                   67
Name: count, dtype: int64
Processed and updated /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv
File: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_2.csv
attack_ca

### Extracting meta data from provided csvs (labeled csv)

In [1]:
from label.parallel_label import *
if __name__ == '__main__':
    #set_start_method("fork")  # or "spawn" depending on your system

    # Example usage:
    folders = ["/scratch/user/syedwali/Datasets/UNSW/provided_CSVs"]

    # Step 1: Extract time ranges from CSV files in all folders using multiple cores
    meta_data = extract_time_ranges_from_csvs(folders, timestamp_column='timestamp', timezone='None', batch_size=5)

    # Print resulting metadata
    print(meta_data)

INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_4.csv: Min timestamp = 1424249984000, Max timestamp = 1424262068000
INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv: Min timestamp = 1424231099000, Max timestamp = 1424250009000
INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_2.csv: Min timestamp = 1421955795000, Max timestamp = 1424231129000
INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv: Min timestamp = 1421927377000, Max timestamp = 1421955842000


                                           file_path  min_timestamp  \
0  /scratch/user/syedwali/Datasets/UNSW/provided_...  1421927377000   
1  /scratch/user/syedwali/Datasets/UNSW/provided_...  1424231099000   
2  /scratch/user/syedwali/Datasets/UNSW/provided_...  1421955795000   
3  /scratch/user/syedwali/Datasets/UNSW/provided_...  1424249984000   

   max_timestamp  
0  1421955842000  
1  1424250009000  
2  1424231129000  
3  1424262068000  


### Labeling

We are now labeling our processed CSV files using the labeled CSVs from UNSW. Since UNSW uses Unix timestamps, we are setting the timezone to None. Additionally, Unix timestamps have second-level precision, which we are considering. In the UNSW dataset, the labels are in the column named attack_cat, which we have also defined as function attributes.

In [2]:
import warnings
from label.parallel_label import *
# Suppress all warnings
warnings.filterwarnings("ignore")
label_csvs(input_folder="/scratch/user/syedwali/Datasets/UNSW/processed/", time_ranges_df=meta_data, timezone='None',num_workers=4,
          unit='sec',flowduration_col='duration',label_col='attack_cat')

Processing CSVs:   0%|          | 0/79 [00:00<?, ?it/s]

Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv


Processing CSVs:   1%|▏         | 1/79 [01:52<2:26:03, 112.35s/it]

Finished processing 11_1000000075_20240905-205247_processed.csv.
Label distribution:
label
Benign    18001
Name: count, dtype: int64


Processing CSVs:   3%|▎         | 2/79 [01:53<1:00:34, 47.21s/it] 

...

Finished processing 19_processed.csv.
Label distribution:
label
Benign              33806
Exploits              793
 Fuzzers              619
 Reconnaissance       339
Generic               126
DoS                   114
 Shellcode             46
Backdoor               17
Worms                   4
No Match                3
Name: count, dtype: int64


Processing CSVs:  99%|█████████▊| 78/79 [1:14:58<00:36, 36.43s/it]

Finished processing 9_1000001414_20240905-212854_processed.csv.
Label distribution:
label
Benign            17489
 Fuzzers            210
Exploits            207
Reconnaissance      104
Generic              36
DoS                  29
Shellcode            14
Backdoors             1
Worms                 1
Name: count, dtype: int64


Processing CSVs: 100%|██████████| 79/79 [1:23:53<00:00, 63.71s/it] 

Finished processing 16_processed.csv.
Label distribution:
label
Benign              39330
Exploits              988
 Fuzzers              720
 Reconnaissance       414
Generic               162
DoS                   134
 Shellcode             53
Backdoor               17
Worms                   8
No Match                2
Analysis                1
Name: count, dtype: int64


Reading labeled CSV files from the UNSW dataset folder. Due to the dataset's large size, each CSV file is read individually, with columns related to ports, IPs, MAC addresses, and payloads being dropped. Additionally, the label count for each file is limited to a maximum of 5000. Unmatched rows have been removed, and the resulting DataFrame is exported. Users can adjust these settings based on their system requirements.

In [3]:
import os
import pandas as pd
from tqdm import tqdm

# Function to process all CSV files in a folder
def process_csvs_in_folder(input_folder):
    # Initialize an empty DataFrame to store the combined results
    combined_df = pd.DataFrame()

    # Iterate over all CSV files in the input folder with progress bar
    csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]
    
    for csv_file in tqdm(csv_files, desc="Processing CSV files"):
        try:
            # Read the CSV file
            file_path = os.path.join(input_folder, csv_file)
            df = pd.read_csv(file_path, low_memory=False)
            
            # Drop unnecessary columns
            df.drop(['id','expiration_id','src_ip','src_mac','src_oui','src_port','dst_ip','dst_mac','dst_oui','dst_port',
                     'protocol','ip_version','vlan_id','tunnel_id','bidirectional_first_seen_ms','bidirectional_last_seen_ms',
                     'src2dst_first_seen_ms','src2dst_last_seen_ms','dst2src_first_seen_ms','dst2src_last_seen_ms',
                     'application_name', 'application_category_name', 'application_is_guessed', 'application_confidence',
                     'requested_server_name','client_fingerprint','server_fingerprint','user_agent','content_type',
                     'udps.payload_data','udps.delta_time','udps.packet_direction','udps.ip_size','udps.transport_size',
                     'udps.payload_size','udps.syn','udps.cwr','udps.ece','udps.urg','udps.ack','udps.psh','udps.rst',
                     'udps.fin','file'], axis=1, inplace=True)

            # Drop rows with missing values
            df.dropna(inplace=True)

            # Remove rows where the label is 'No Match'
            df = df[df['label'] != 'No Match']

            # Limit the number of instances per class label to 5000
            df = df.groupby('label').apply(lambda x: x.head(5000)).reset_index(drop=True)

            # Append the processed DataFrame to the combined DataFrame
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        
        except Exception as e:
            print(f"Failed to process {csv_file}: {e}")

    # Return the combined DataFrame
    return combined_df

# Example usage:
input_folder = "/scratch/user/syedwali/Datasets/UNSW/processed/labeled_csv"
df = process_csvs_in_folder(input_folder)


Processing CSV files: 100%|██████████| 78/78 [23:05<00:00, 17.77s/it]


In [4]:
df['label'] = df['label'].str.strip()

In [5]:
df['label'].value_counts()

label
Benign            390000
Exploits           25158
Fuzzers            18199
Reconnaissance     11086
Generic             3861
DoS                 3642
Shellcode           1433
Analysis             426
Backdoor             340
Worms                156
Backdoors             50
Name: count, dtype: int64

In [6]:
# If needed, save the combined DataFrame to a CSV
output_file = "/scratch/user/syedwali/Datasets/UNSW_dataset.csv"
df.to_csv(output_file, index=False)

In [12]:
df=pd.read_csv("/scratch/user/syedwali/Datasets/UNSW_dataset.csv")

In [13]:
df['label'] = df['label'].replace(['Backdoors', 'Backdoor'], 'Backdoor')

In [14]:
df['label'].value_counts()

label
Benign            390000
Exploits           25158
Fuzzers            18199
Reconnaissance     11086
Generic             3861
DoS                 3642
Shellcode           1433
Analysis             426
Backdoor             390
Worms                156
Name: count, dtype: int64

In [15]:
df.drop(['flowid'],axis=1,inplace=True)

In [8]:
df.columns

Index(['bidirectional_duration_ms', 'bidirectional_packets',
       'bidirectional_bytes', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_duration_ms', 'dst2src_packets',
       'dst2src_bytes', 'bidirectional_min_ps', 'bidirectional_mean_ps',
       'bidirectional_stddev_ps', 'bidirectional_max_ps', 'src2dst_min_ps',
       'src2dst_mean_ps', 'src2dst_stddev_ps', 'src2dst_max_ps',
       'dst2src_min_ps', 'dst2src_mean_ps', 'dst2src_stddev_ps',
       'dst2src_max_ps', 'bidirectional_min_piat_ms',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'src2dst_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms', 'src2dst_max_piat_ms',
       'dst2src_min_piat_ms', 'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'bidirectional_syn_packets',
       'bidirectional_cwr_packets', 'bidirectional_ece_packets',
       'bidirectional_urg_packets', 'bidirectional_ack_p

### Performance Evaluation with contextual Features

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming df is already defined and has a 'label' column
X = df.drop('label', axis=1)  # Features (drop the label column)
y = df['label']  # Labels

# Split the data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.98
Classification Report:
                precision    recall  f1-score   support

      Analysis       0.80      0.82      0.81       128
      Backdoor       0.83      0.57      0.68       112
        Benign       1.00      1.00      1.00    117065
           DoS       0.71      0.26      0.38      1075
      Exploits       0.84      0.92      0.88      7531
       Fuzzers       0.92      0.95      0.93      5408
       Generic       0.88      0.72      0.79      1164
Reconnaissance       0.91      0.91      0.91      3363
     Shellcode       0.63      0.62      0.62       423
         Worms       0.64      0.43      0.52        37

      accuracy                           0.98    136306
     macro avg       0.82      0.72      0.75    136306
  weighted avg       0.98      0.98      0.98    136306



### Performance Evaluation without contextual Features

To demonstrate the impact of extended contextual features on classification performance, we've removed all such features from our analysis. By doing so, we can observe the potential degradation in classification accuracy when contextual information is limited.

In [17]:
df.drop(['udps.srcdst_packet_size_variation','udps.srcdst_udp_packet_count','udps.udp_packet_count',
'udps.srcdst_tcp_packet_count','udps.tcp_packet_count','udps.srcdst_ack_packet_count',
'udps.ack_packet_count','udps.srcdst_fin_packet_count','udps.fin_packet_count',
'udps.srcdst_rst_packet_count','udps.rst_packet_count','udps.srcdst_psh_packet_count',
'udps.psh_packet_count','udps.srcdst_syn_packet_count','udps.syn_packet_count','udps.srcdst_unique_ports_count',
'udps.srcdst_icmp_packet_count','udps.icmp_packet_count','udps.srcdst_http_ports_count','udps.http_ports_count',
'udps.srcdst_bidirectional_duration_avg','udps.bidirectional_duration_avg','udps.srcdst_dns_port_count',
'udps.dns_port_count','udps.srcdst_dns_port_src_count','udps.dns_port_src_count','udps.srcdst_vul_ports_count',
'udps.src2dst_packet_count','udps.bidirectional_packet_count','udps.srcdst_src2dst_packet_count',
'udps.srcdst_bidirectional_packet_count'],axis=1,inplace=True)

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming df is already defined and has a 'label' column
X = df.drop('label', axis=1)  # Features (drop the label column)
y = df['label']  # Labels

# Split the data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.97
Classification Report:
                precision    recall  f1-score   support

      Analysis       0.38      0.27      0.31       128
      Backdoor       0.86      0.58      0.69       112
        Benign       1.00      0.99      0.99    117065
           DoS       0.70      0.36      0.48      1075
      Exploits       0.83      0.91      0.86      7531
       Fuzzers       0.79      0.86      0.83      5408
       Generic       0.87      0.79      0.83      1164
Reconnaissance       0.90      0.90      0.90      3363
     Shellcode       0.58      0.53      0.56       423
         Worms       0.61      0.54      0.57        37

      accuracy                           0.97    136306
     macro avg       0.75      0.67      0.70    136306
  weighted avg       0.97      0.97      0.97    136306

